<a href="https://colab.research.google.com/github/ee17b031-iittp/Projected-Gradient-Descent-with-CIFAR10/blob/main/pgd_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from imageio import imwrite
import cv2

In [2]:
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [3]:
(ds_train,ds_validation, ds_test),ds_info = tfds.load("cifar10",
                                       split=['train[:99%]','train[99%:]','test'],
                                       shuffle_files=True,
                                       as_supervised = True,
                                       with_info = True)

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteCQ08QA/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteCQ08QA/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [4]:
img_height=128
img_width=128
batch_size=16

In [5]:
def preprocess(img,label):
  imag=tf.image.resize(img,size=[img_height,img_width])
  return tf.divide(tf.cast(imag,tf.float32), tf.constant(255.0)),tf.one_hot(label,depth=10)

ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.map(preprocess,
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache('./traindata')
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
ds_validation = ds_validation.batch(batch_size)
ds_validation = ds_validation.map(
    preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_validation = ds_validation.cache('./valdata')
ds_validation = ds_validation.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.map(
    preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.cache('./testdata')
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
inputs = tf.keras.layers.Input(shape=(img_height, img_width, 3))
res50 = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=inputs)
x = res50.layers[-20].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
output_layer = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, output_layer)

102973440/102967424 [==============================] - 1s 0us/step


In [9]:
for layer in res50.layers:
    if isinstance(layer,tf.keras.layers.BatchNormalization):
      layer.trainable=True
    else:
      layer.trainable=False
model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='categorical_crossentropy',
        metrics=['accuracy'])

In [10]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [11]:
history = model.fit(ds_train, validation_data=ds_validation,verbose=1, epochs=20)

Epoch 1/20
3094/3094 [==============================] - 221s 71ms/step - loss: 0.5503 - accuracy: 0.8154 - val_loss: 0.2846 - val_accuracy: 0.8980
Epoch 2/20
3094/3094 [==============================] - 271s 88ms/step - loss: 0.2638 - accuracy: 0.9112 - val_loss: 0.2992 - val_accuracy: 0.8980
Epoch 3/20
3094/3094 [==============================] - 181s 58ms/step - loss: 0.1370 - accuracy: 0.9544 - val_loss: 0.3941 - val_accuracy: 0.8940
Epoch 4/20
3094/3094 [==============================] - 181s 58ms/step - loss: 0.0839 - accuracy: 0.9706 - val_loss: 0.4492 - val_accuracy: 0.8980
Epoch 5/20
3094/3094 [==============================] - 180s 58ms/step - loss: 0.0681 - accuracy: 0.9749 - val_loss: 0.4680 - val_accuracy: 0.9000
Epoch 6/20
3094/3094 [==============================] - 184s 59ms/step - loss: 0.0512 - accuracy: 0.9822 - val_loss: 0.5516 - val_accuracy: 0.9060
Epoch 7/20
3094/3094 [==============================] - 186s 60ms/step - loss: 0.0440 - accuracy: 0.9841 - val_loss: 0

In [12]:
iterations = 7
alpha = 2
epsilon = 8/255

In [25]:
#pgd_generation
image_count = 0
flag = 0
orig_label = []
for x_train, labels in ds_train:
  gen_img = tf.identity(x_train)
  gen_img = gen_img + tf.random.uniform(gen_img.get_shape().as_list(), minval=-epsilon, maxval=epsilon, dtype=tf.dtypes.float32)
  x_temp = x_train.numpy()
  for iters in range(iterations):
    imgv = tf.Variable(gen_img)
    with tf.GradientTape() as tape:
      tape.watch(imgv)
      predictions = model(imgv)
      loss = tf.keras.losses.CategoricalCrossentropy()(labels, predictions)
      grads = tape.gradient(loss,imgv)
    signed_grads = tf.sign(grads)
    gen_img = gen_img + (alpha*signed_grads)
    gen_img = tf.clip_by_value(gen_img, x_train-epsilon, x_train+epsilon)
  for i in range(16):
    image_count += 1
    if image_count>200:
      flag = 1
      break
    orig_label.append(labels[i])
    gen_image = gen_img.numpy()[i]*255
    orig_image = x_train.numpy()[i]*255
    imwrite('/content/drive/My Drive/pgd_cifar10/'+str(image_count)+'_gen_img.png',np.clip(gen_image, 0, 255).astype('uint8'))
    imwrite('/content/drive/My Drive/pgd_cifar10/'+str(image_count)+'_orig_img.png',np.clip(orig_image, 0, 255).astype('uint8'))
  if flag==1:
    break 

In [27]:
generated_images = []
for i in range(1, 201):
  x = cv2.imread('/content/drive/My Drive/pgd_cifar10/'+str(i)+'_gen_img.png')
  generated_images.append(x)
generated_images = tf.convert_to_tensor(generated_images)
orig_label = tf.convert_to_tensor(orig_label)

In [31]:
original_images = []
for i in range(1, 201):
  x = cv2.imread('/content/drive/My Drive/pgd_cifar10/'+str(i)+'_orig_img.png')
  original_images.append(x)
original_images = tf.convert_to_tensor(generated_images)

In [28]:
model.evaluate(generated_images,orig_label, verbose = 1)

7/7 [==============================] - 0s 49ms/step - loss: 6225.9824 - accuracy: 0.0900


[6225.982421875, 0.09000000357627869]

In [30]:
def calculate_fid(images1, images2):
  incp_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
  # calculate activations
  act1 = incp_model.predict(images1)
  act2 = incp_model.predict(images2)
  # calculate mean and covariance statistics
  mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
  mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
  # calculate sum squared difference between means
  ssdiff = np.sum((mu1 - mu2)**2.0)
  # calculate sqrt of product between cov
  covmean = sqrtm(sigma1.dot(sigma2))
  # check and correct imaginary numbers from sqrt
  if np.iscomplexobj(covmean):
    covmean = covmean.real
  # calculate score
  fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
  return fid

In [32]:
original_images=tf.image.resize(original_images,size=[299,299])
generated_images = tf.image.resize(generated_images,size=[299,299])

In [36]:
print("fid =", calculate_fid(original_images, generated_images))#Frechet Inception Distance

87916544/87910968 [==============================] - 1s 0us/step
fid = -0.03991262250921064
